In [3]:
using DifferentialEquations
using Plots

gr(size=(1000, 800), dpi=300, legendfontsize=12, tickfontsize=10, guidefontsize=12)

# Параметры для моего варианта
p_cr = 11.5    # критическая стоимость (тыс.ед.)
N = 35         # число потребителей (тыс.чел)
q = 1          # максимальная потребность

# Фирма 1
tau1 = 18      # длительность производственного цикла
p_tilde1 = 7.8 # себестоимость (тыс.ед.)
M10 = 4.8      # начальные оборотные средства (млн.ед.)

# Фирма 2
tau2 = 28      # длительность производственного цикла
p_tilde2 = 5.7 # себестоимость (тыс.ед.)
M20 = 4.3      # начальные оборотные средств (млн.ед.)

# Расчет коэффициентов
a1 = p_cr / (tau1^2 * p_tilde1^2 * N * q)
a2 = p_cr / (tau2^2 * p_tilde2^2 * N * q)
b = p_cr / (tau1^2 * tau2^2 * p_tilde1^2 * p_tilde2^2 * N * q)
c1 = (p_cr - p_tilde1) / (tau1 * p_tilde1)
c2 = (p_cr - p_tilde2) / (tau2 * p_tilde2)

println("Рассчитанные коэффициенты:")
println("a1 = ", round(a1, digits=6))
println("a2 = ", round(a2, digits=6))
println("b = ", round(b, digits=9))
println("c1 = ", round(c1, digits=6))
println("c2 = ", round(c2, digits=6))

# Случай 1: Только рыночная конкуренция
function case1!(du, u, p, t)
    M1, M2 = u
    du[1] = M1 - (b/c1)*M1*M2 - (a1/c1)*M1^2
    du[2] = (c2/c1)*M2 - (b/c1)*M1*M2 - (a2/c1)*M2^2
end

# Случай 2: С социально-психологическими факторами
function case2!(du, u, p, t)
    M1, M2 = u
    du[1] = M1 - (b/c1 + 0.0004)*M1*M2 - (a1/c1)*M1^2
    du[2] = (c2/c1)*M2 - (b/c1)*M1*M2 - (a2/c1)*M2^2
end

# Временной интервал для безразмерного времени theta
tspan = (0.0, 30.0)
u0 = [M10, M20]

# Решение систем
prob1 = ODEProblem(case1!, u0, tspan)
prob2 = ODEProblem(case2!, u0, tspan)

sol1 = solve(prob1, Tsit5(), reltol=1e-8, abstol=1e-8)
sol2 = solve(prob2, Tsit5(), reltol=1e-8, abstol=1e-8)

# Построение графиков
p1 = plot(sol1, label=["Фирма 1" "Фирма 2"], 
          xlabel="Безразмерное время θ", ylabel="Оборотные средства M (млн.ед.)",
          title="Случай 1: Только рыночная конкуренция",
          linewidth=2, grid=true, legend=:right)

p2 = plot(sol2, label=["Фирма 1" "Фирма 2"], 
          xlabel="Безразмерное время θ", ylabel="Оборотные средства M (млн.ед.)",
          title="Случай 2: С социально-психологическими факторами",
          linewidth=2, grid=true, legend=:right)

# Сохранение графиков
savefig(p1, "competition_case1.png")
savefig(p2, "competition_case2.png")

# Вывод стационарных состояний для случая 1
println("\nСтационарные состояния для случая 1:")
println("Фирма 1 конечное значение: ", round(sol1.u[end][1], digits=3))
println("Фирма 2 конечное значение: ", round(sol1.u[end][2], digits=3))

# Анализ результатов
println("\nАнализ результатов:")
println("Случай 1: Обе фирмы достигают стабильного состояния")
println("Случай 2: Влияние социально-психологических факторов изменяет динамику")

# Дополнительный анализ стационарных точек
# Для случая 1: решаем систему уравнений dM/dθ = 0
function stationary_points()
    # Уравнения: M1*(1 - (b/c1)*M2 - (a1/c1)*M1) = 0
    #           M2*((c2/c1) - (b/c1)*M1 - (a2/c1)*M2) = 0
    
    # Тривиальное решение
    println("\nСтационарные точки:")
    println("1. M1 = 0, M2 = 0")
    
    # Нетривиальные решения
    # Из первого уравнения: 1 - (b/c1)*M2 - (a1/c1)*M1 = 0
    # Из второго уравнения: (c2/c1) - (b/c1)*M1 - (a2/c1)*M2 = 0
    
    # Решаем систему линейных уравнений
    A = [a1/c1 b/c1; b/c1 a2/c1]
    B = [1; c2/c1]
    
    M_stationary = A \ B
    println("2. M1 = ", round(M_stationary[1], digits=3), ", M2 = ", round(M_stationary[2], digits=3))
end

stationary_points()

Рассчитанные коэффициенты:
a1 = 1.7e-5
a2 = 1.3e-5
b = 1.0e-9
c1 = 0.026353
c2 = 0.036341

Стационарные состояния для случая 1:
Фирма 1 конечное значение: 1580.915
Фирма 2 конечное значение: 2817.207

Анализ результатов:
Случай 1: Обе фирмы достигают стабильного состояния
Случай 2: Влияние социально-психологических факторов изменяет динамику

Стационарные точки:
1. M1 = 0, M2 = 0
2. M1 = 1580.915, M2 = 2817.207
